In [19]:
import os
import pandas as pd
import numpy as np

# Make Mergered Data

In [20]:
# 기본 경로 설정
base_dir = 'Subjects'
subject_prefix = 'AB'
subfolders = ['levelground', 'ramp', 'stair']

In [33]:
# Subjects 폴더 내 모든 서브젝트 폴더 탐색
subject_dirs = [d for d in os.listdir(base_dir) if d.startswith(subject_prefix)]

# 각 서브젝트 폴더 순회
for subject in subject_dirs:
    subject_dir = os.path.join(base_dir, subject)
    
    for subfolder in subfolders:
        conditions_dir = os.path.join(subject_dir, subfolder, 'conditions')
        imu_dir = os.path.join(subject_dir, subfolder, 'imu')
        
        if not os.path.exists(conditions_dir) or not os.path.exists(imu_dir):
            continue
        
        # 조건 폴더 내 모든 CSV 파일 탐색
        condition_files = [f for f in os.listdir(conditions_dir) if f.endswith('.csv')]
        
        for condition_file in condition_files:
            condition_file_path = os.path.join(conditions_dir, condition_file)
            imu_file_path = os.path.join(imu_dir, condition_file.replace('_labels', ''))
            
            if not os.path.exists(imu_file_path):
                print(f'Matching IMU file not found for {condition_file_path}')
                continue
            
            # CSV 파일 로드
            condition_df = pd.read_csv(condition_file_path)
            imu_df = pd.read_csv(imu_file_path)
            
            # 'Header' 열 이름을 'time'으로 변경
            condition_df = condition_df.rename(columns={'Header': 'time'})
            imu_df = imu_df.rename(columns={'Header': 'time'})
            
            # 'time' 열을 기준으로 데이터 병합
            merged_df = pd.merge(condition_df, imu_df, on='time', how='inner')
            
            # 결과 출력
            # print(f'Merged DataFrame for {condition_file_path} and {imu_file_path}:\n', merged_df.head())
            
            # 필요한 경우, 병합된 데이터를 새로운 CSV 파일로 저장
            output_dir = os.path.join(subject_dir, subfolder, 'train_data__IMU')
            os.makedirs(output_dir, exist_ok=True)
            output_file_path = os.path.join(output_dir, f'{condition_file}')
            merged_df.to_csv(output_file_path, index=False)


# Make X & Y

In [21]:
# 데이터를 저장할 리스트 초기화
X_list = []
Y_list = []

# Subjects 폴더 내 모든 서브젝트 폴더 탐색
subject_dirs = [d for d in os.listdir(base_dir) if d.startswith(subject_prefix)]

# 각 서브젝트 폴더 순회
for subject in subject_dirs:
    subject_dir = os.path.join(base_dir, subject)
    
    for subfolder in subfolders:
        merged_dir = os.path.join(subject_dir, subfolder, 'train_data__IMU')
        
        if not os.path.exists(merged_dir):
            continue
        
        # 병합된 데이터 폴더 내 모든 CSV 파일 탐색
        merged_files = [f for f in os.listdir(merged_dir) if f.endswith('.csv')]
        
        for merged_file in merged_files:
            merged_file_path = os.path.join(merged_dir, merged_file)
            
            # CSV 파일 로드
            df = pd.read_csv(merged_file_path)
            
            # 'time' 열 제거
            if 'time' in df.columns:
                df = df.drop(columns=['time'])
            
            # 'Label' 열을 Y로 설정하고 나머지 열을 X로 설정
            if 'Label' in df.columns:
                X = df.drop(columns=['Label'])
                Y = df['Label']
            else:
                print(f'Label column not found in {merged_file_path}')
                continue
            
            # 데이터를 리스트에 추가
            X_list.append(X)
            Y_list.append(Y)

# 리스트를 numpy 배열로 변환 (dtype=object 지정)
X_data = np.array(X_list, dtype=object)
Y_data = np.array(Y_list, dtype=object)

# 결과 출력 (형태 확인)
print('X_data shape:', X_data.shape)
print('Y_data shape:', Y_data.shape)

# 데이터를 파일로 저장
np.save('X_data.npy', X_data)
np.save('Y_data.npy', Y_data)

X_data shape: (2990,)
Y_data shape: (2990,)
